## Installing Necessary Libraries

In [1]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.4 MB/s eta 0:00:00


# Getting data

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
data_train = pd.read_csv('mts_ai_data_164_tickets.csv')

In [4]:
data_train

,text,data
0,"Привет, меня зовут Иванов Иван Петрович, возьм...","{'Имя': ['Иванов Иван Петрович'], 'Количество ..."
1,"Привет, я Кузнецова Ольга Валерьевна, нужны би...","{'Имя': ['Кузнецова Ольга Валерьевна'], 'Колич..."
2,Привет! Меня зовут Смирнов Алексей Николаевич....,"{'Имя': ['Смирнов Алексей Николаевич'], 'Колич..."
3,Добрый вечер! Меня зовут Попова Екатерина Дмит...,"{'Имя': ['Чернов Максим Алексеевич', 'Степанов..."
4,Привет! Меня зовут Козлова Маргарита Александр...,"{'Имя': ['Козлова Маргарита Александровна'], '..."
...,...,...
159,Здравствуй! Меня зовут Власов Валентин Анатоль...,"{'Имя': ['Власов Валентин Анатольевич'], 'Коли..."
160,Hello! My name is Abramov Dmitry Vladimirovich...,"{'Имя': ['Abramov Dmitry Vladimirovich'], 'Кол..."
161,Привет! Меня зовут Thompson John Michael. С се...,"{'Имя': ['Thompson John Michael', 'Thompson Je..."
162,Good day! I'm Galkin Ivan Petrovich. I'm flyin...,"{'Имя': ['Galkin Ivan Petrovich'], 'Количество..."


In [5]:
from datasets import Dataset,DatasetDict
train_dataset_dict = DatasetDict({
    "train": Dataset.from_pandas(data_train),
})

## Loading the model and training


In [ ]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [7]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = 'meta-llama/Llama-2-7b-chat-hf'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    use_nested_quant = False
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map = {"": 0}
    # trust_remote_code=True
)

model.config.use_cache = False

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [10]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRM

In [11]:
import re
pattern = r'\((\w+)\): Linear'
linear_layers = re.findall(pattern, str(model.modules))
target_modules = list(set(linear_layers))
print(target_modules)

['v_proj', 'lm_head', 'gate_proj', 'q_proj', 'up_proj', 'k_proj', 'o_proj', 'down_proj']


In [12]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias = 'none',
    task_type="CAUSAL_LM",
    target_modules=['q_proj','v_proj']
)

In [13]:
from transformers import TrainingArguments

warmup_ratio = 0.03
output_dir = "./llama2-7b_distNER_mts"
per_device_train_batch_size = 1
gradient_accumulation_steps = 1
optim = "paged_adamw_32bit"
save_steps = 50
num_train_epochs=5
logging_steps = 10
learning_rate = 4e-5
max_grad_norm = 0.3
warmup_ratio = 0.03
lr_scheduler_type = "cosine"
weight_decay = 0.001

training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    gradient_checkpointing=True,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    max_grad_norm=max_grad_norm,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    fp16 = False,
    bf16 = False,
    lr_scheduler_type=lr_scheduler_type,
)

In [14]:
b_inst, e_inst = "[INST]", "[/INST]"
b_sys, e_sys = "<<SYS>>\n", "\n<</SYS>>\n\n"
def_system_prompt = 'You are very helpful information extraction system.'

system_prompt = b_sys + def_system_prompt + e_sys

def formatting_prompts_func(data_train):
    output_texts = []
    for i in range(len(data_train['text'])):
        text = f'''Given an input, your task is to extract entities Name (Имя), Number of passengers (Количество пассажиров), Departure city (Город вылета), Arrival city (Город прилета), Time (Время), Date (Дата), Airline (Авиакомпания).
        Input: {data_train['text'][i]}
        The output should be in a dictionary: {data_train['data'][i]}.
        '''

        result_prompt = b_inst + system_prompt + text + e_inst
        output_texts.append(result_prompt)
    return output_texts

In [ ]:
# def formatting_prompts_func(data_train):
#     output_texts = []
#     for i in range(len(data_train['text'])):
#         text = f'''System message: You are very helpful information extraction system.
#         Given a passage, your task is to extract entities Name (Имя), Number of passengers (Количество пассажиров), Departure city (Город вылета), Arrival city (Город прилета), Time (Время), Date (Дата), Airline (Авиакомпания).
#         The output should be in a dictionary of the following format: {data_train['data'][i]}.
#         Passage: {data_train['text'][i]}'''
#         output_texts.append(text)
#     return output_texts

In [15]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_dict['train'],
    peft_config=peft_config,
    max_seq_length = None,
    tokenizer=tokenizer,
    args=training_arguments,
    formatting_func=formatting_prompts_func
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:222: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/164 [00:00<?, ? examples/s]

In [16]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [ ]:
#import gc
#gc.collect()
#torch.cuda.empty_cache()

In [17]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,1.942400
20,1.974600
30,1.860500
40,1.523500
50,1.292300
60,1.152000
70,0.981100
80,0.806200
90,0.720500
100,0.555000


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

Step,Training Loss
10,1.942400
20,1.974600
30,1.860500
40,1.523500
50,1.292300
60,1.152000
70,0.981100
80,0.806200
90,0.720500
100,0.555000


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

TrainOutput(global_step=820, training_loss=0.3631613218202824, metrics={'train_runtime': 3837.7206, 'train_samples_per_second': 0.214, 'train_steps_per_second': 0.214, 'total_flos': 9480612462305280.0, 'train_loss': 0.3631613218202824, 'epoch': 5.0})

In [22]:
text = 'Привет! Меня зовут Моисеенко Олеся Игоревна и я хочу купить билеты на самолет в Рим из Санкт_Петербурга на рейс в 12:30 на 25 января 2024 года. Полечу я компанией Аэрофлот'

In [23]:
text_new = f'''Given a passage, your task is to extract entities Name (Имя), Number of passengers (Количество пассажиров), Departure city (Город вылета), Arrival city (Город прилета), Time (Время), Date (Дата), Airline (Авиакомпания).
        Passage: {text}
        The output should be in a dictionary format.'''

result_text = b_inst + system_prompt + text_new + e_inst

In [24]:
enc = tokenizer(text, padding=True, return_tensors='pt')
enc = enc.to('cuda')
out = model.generate(input_ids=enc['input_ids'], attention_mask=enc['attention_mask'], max_length=500, pad_token_id=50256)  # max_length=200, temperature=0.5, repetition_penalty=2.0,
print(tokenizer.batch_decode(out))

['<s> Привет! Меня зовут Моисеенко Олеся Игоревна и я хочу купить билеты на самолет в Рим из Санкт_Петербурга на рейс в 12:30 на 25 января 2024 года. Полечу я компанией Аэрофлот. Спасибо!\nВы можете купить билеты для меня:\n\n1. Моисеенко Олеся Игоревна\n2. Моисеенко Олег Игоревич (другой пассажир)\n\nНеобходимо забронировать дополнительное место в багаже. С удовольствием приобречу билеты.</s>']


In [18]:
trainer.push_to_hub()

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1703703796.fa2048ad068d.287.0:   0%|          | 0.00/18.0k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/134M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.73k [00:00<?, ?B/s]

'https://huggingface.co/olesya2096/llama2-7b_distNER_mts/tree/main/'